In [1]:
from facebook_scraper import get_posts
import json
import pandas as pd
import twitter
import internetarchive as ia
import requests
import os
import datetime as dt
from git import Repo
import warnings
import locale
locale.setlocale(locale.LC_TIME, 'es_US.UTF8');

In [2]:
datos_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSYiPWLCf9_HWB51cIryodmVlDg4RQS_AeccuJjkz9xVtDx7Y5tfk9RaF94BrVdO5vakBo5dUktZawh/pub?gid=0&single=true&output=csv"
metadatos_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSYiPWLCf9_HWB51cIryodmVlDg4RQS_AeccuJjkz9xVtDx7Y5tfk9RaF94BrVdO5vakBo5dUktZawh/pub?gid=1875795980&single=true&output=csv"
directory = 'reportes'
casos_acumulados_url = 'https://raw.githubusercontent.com/sociedatos/covid19-bo-casos_por_departamento/master/confirmados_acumulados.csv'

In [3]:
def load_credenciales():
    with open('credenciales.json', 'r') as f:
        return json.load(f)
    
def auth_twitter():
    "Autenticar en Twitter para realizar consultas al API"

    return twitter.Api(consumer_key = credenciales['twitter']['consumer_key'],
                       consumer_secret = credenciales['twitter']['consumer_secret'],
                       access_token_key = credenciales['twitter']['access_token_key'],
                       access_token_secret = credenciales['twitter']['access_token_secret'],
                       tweet_mode='extended')

def read_metadata():
    "Descarga los metadatos de google sheets, filtra sólo aquellos con una fuente y crea una columna de status_id"

    meta = pd.read_csv(metadatos_url, parse_dates=['fecha'], index_col=['fecha'])
    meta = meta.dropna(subset=['La Paz', 'Cochabamba', 'Oruro'])
    return meta

def filtrar_nuevos(meta):
    "Devuelve sólo las entradas que no hayan sido descargadas previamente."

    viejos = [dt.datetime.strptime(filename.split('.')[0].split('_')[-1], '%Y%m%d') for filename in os.listdir(directory)]
    return meta[~meta.index.isin(viejos)]

def create_identifier(dep, fecha):
    return 'reportesdepartamentalescovid19bolivia_{}_{}'.format(dep.lower().replace(' ', '_'), fecha.strftime('%Y%m%d'))

def save_image(url, filename):
    with open(filename, 'wb') as f:
        f.write(requests.get(url).content)

def parse_facebook(url, dep, fecha):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        post = [post for post in get_posts(post_urls=[url], cookies='cookies.json')][0]
    content_type = requests.head(post['images'][0]).headers.get('content-type')
    is_image = 'image' in content_type
    post_meta = {
        "source": post['post_url'],
        "creator": post['username'],
        "description": post['text'],
    }
    identifier = create_identifier(dep, fecha)
    filename = '{}/{}.jpeg'.format(directory, identifier)
    if is_image:
        save_image(post['images'][0], filename)
    return is_image, post_meta, identifier, filename

def parse_twitter(url, dep, fecha):
    tweet_id = url.split('/')[-1]
    statuses = tw.GetStatuses([tweet_id])
    status = statuses[0]._json
    post_meta = {
        "source": url,
        "creator": status['user']['name'],
        "description": status['full_text']
    }
    identifier = create_identifier(dep, fecha)
    media = status['entities']['media'][0]['media_url']
    filename = '{}/{}.{}'.format(directory, identifier, media.split('.')[-1])
    save_image(media, filename)
    return True, post_meta, identifier, filename

def upload_archive(dep, fecha, post_meta, identifier, filename):

    ia_meta = {'title': 'Reporte de Covid-19 en {}, Bolivia para el {}'.format(dep, fecha.strftime('%-d de %B, %Y')),
               'description': post_meta['description'],
               'source': post_meta['source'],
               'collection': 'covid19-bolivia-departamentos',
               'creator': post_meta['creator'],
               'mediatype': 'image',
               'date': fecha.strftime('%Y-%m-%d')}

    ia.upload(identifier,
              filename,
              metadata=ia_meta,
              access_key = credenciales["ia"]["access"],
              secret_key = credenciales["ia"]["secret"],
              retries=4
             )
    print('https://archive.org/details/{} : {}'.format(identifier, post_meta['source']))

def archivar(meta):
    for fecha, row in meta.iterrows():
        valores = row.to_dict()
        for dep in valores.keys():
            if type(valores[dep]) == str:
                url = valores[dep]
                if 'facebook.com' in url:
                    is_image, post_meta, identifier, filename = parse_facebook(url, dep, fecha)
                if 'twitter.com' in url:
                    is_image, post_meta, identifier, filename = parse_twitter(url, dep, fecha)
                if is_image:
                    upload_archive(dep, fecha, post_meta, identifier, filename)

In [4]:
credenciales = load_credenciales()
tw = auth_twitter()
meta = filtrar_nuevos(read_metadata())
archivar(meta)

https://archive.org/details/reportesdepartamentalescovid19bolivia_la_paz_20220321 : https://facebook.com/story.php?story_fbid=5154111557943168&id=1140266705994360
https://archive.org/details/reportesdepartamentalescovid19bolivia_cochabamba_20220321 : https://facebook.com/story.php?story_fbid=4950457621715600&id=957131637714905
https://archive.org/details/reportesdepartamentalescovid19bolivia_oruro_20220321 : https://facebook.com/story.php?story_fbid=337484875073843&id=100064370004661
https://archive.org/details/reportesdepartamentalescovid19bolivia_potosi_20220321 : https://facebook.com/story.php?story_fbid=513743276878875&id=110681273851746
https://archive.org/details/reportesdepartamentalescovid19bolivia_tarija_20220321 : https://facebook.com/story.php?story_fbid=1033268950635115&id=396941987601151
https://archive.org/details/reportesdepartamentalescovid19bolivia_chuquisaca_20220321 : https://facebook.com/story.php?story_fbid=808697020533719&id=108534767216618
https://archive.org/det

In [36]:
def consolidar():
    """
    Sincroniza sheets
    """
    
    def download_sheet(url, filename):
        datos = requests.get(url).text
        datos = datos.replace('\r','')
        with open(filename, 'w+') as f:
            f.write(datos)
            
    for url, filename in zip([metadatos_url, datos_url], ['metadata.csv', 'descartados.csv']):
        download_sheet(url, filename)

def format_descartados(dep_cols):
    
    start = '2022-01-21'
    descartados = pd.read_csv('descartados.csv', parse_dates=['Fecha'], index_col=['Fecha'])

    # Llenar el departamento faltante (usualmente Santa Cruz)
    for fecha, row in descartados.loc[start:].iterrows():
        if row[dep_cols].isna().sum() == 1:
            missing = row["Bolivia"] - row[dep_cols].sum()
            if missing > 0:
                row.loc[row[row.isna()].index] = missing
    
    # Descartar datos cuando el valor de descartados diarios es menor a 0
    for dep in dep_cols:
        descartados[dep].loc[descartados[dep].diff() < 0] = None
    
    return descartados

# def format_pruebas(casos, descartados, dep_cols):
    # pruebas = (casos[dep_cols] + descartados[dep_cols].interpolate()).dropna()
    # empty = pd.DataFrame(index=pd.date_range(pruebas.index.min(), pruebas.index.max()), columns=dep_cols)
    # pruebas = pd.concat([pruebas, empty])
    # pruebas = pruebas[~pruebas.index.duplicated()].sort_index()
    # pruebas = pruebas.astype(float).interpolate().diff().dropna().astype(int)
    # return pruebas

def format_results():
    dep_cols = ['La Paz', 'Cochabamba', 'Santa Cruz', 'Oruro', 'Potosi', 'Tarija', 'Chuquisaca', 'Beni', 'Pando']
    descartados = format_descartados(dep_cols)
    casos = pd.read_csv(casos_acumulados_url, parse_dates=[0], index_col=0)
    casos = casos.rename(columns={'Potosí':'Potosi'})
    common_index = casos.index.intersection(descartados.index)
    # pruebas = format_pruebas(casos, descartados, dep_cols)
    pruebas = (casos.loc[common_index][dep_cols] + descartados.loc[common_index][dep_cols]).diff()
    # positividad = (casos.diff().dropna() / pruebas).dropna()
    positividad = casos.loc[common_index][dep_cols].diff() / pruebas.loc[common_index][dep_cols]
    
    # Fix days where no tests were run
    for col in pruebas.columns:
        for i in pruebas.loc[pruebas[col] == 0].index:
            positividad.loc[i, col] = 0
    
    return descartados, pruebas, positividad

def save_results(descartados, pruebas, positividad):
    descartados.to_csv('descartados.csv', float_format="%.0f")
    pruebas.to_csv('pruebas.csv', float_format="%.0f")
    positividad.to_csv('positividad.csv', float_format="%.3f")

In [37]:
consolidar()
descartados, pruebas, positividad = format_results()
save_results(descartados, pruebas, positividad)

In [39]:
def update():
    """
    Actualiza el repositorio
    """
    
    last_update = pd.read_csv('metadata.csv', parse_dates=['fecha']).fecha.max()
    repository = Repo('.')
    repository.git.add(all=True)
    repository.index.commit(last_update.strftime('%Y-%m-%d'))
    repository.remotes.origin.push()

In [12]:
update()